In [84]:
def ksa_shuffle(secret_key):
    # 1) Initialize state array S: ["00","01",...,"FF"]
    S = [f"{i:02X}" for i in range(256)]
    j = 0
    keylen = len(secret_key)
    # 2) KSA loop
    for i in range(256):#256
        print("\ni: ", i )
        # convert S[i] from hex-string to integer
        Si = int(S[i], 16)
        print("Si: ", Si )
        # add S[i], key byte, and previous j, mod 256
        print("secret_key[i % keylen]",secret_key[i % keylen])
        j = (j + Si + secret_key[i % keylen]) & 0xFF
        print("j: ", j )
        # swap S[i] and S[j]
        print("Swap ","Si:", S[i] ," with Sj: ",S[j])
        S[i], S[j] = S[j], S[i]

    i = 0
    j = 0
    for k in range(32):
        i = (i + 1) & 0xFF                # advance i
        Si = int(S[i], 16)               # current S[i] as int
        j = (j + Si) & 0xFF              # advance j by that value
        # swap entries
        S[i], S[j] = S[j], S[i]
    return S



In [53]:

# Example key: bytes [0xAB, 0xCD, 0xEF]
secret_key = [0x00, 0x02, 0x49]
shuffled = ksa_shuffle(secret_key)

print(shuffled)
# # Print the resulting state array
# for idx, val in enumerate(shuffled):
#     print(f"{idx:02X}: {val}")



i:  0
Si:  0
secret_key[i % keylen] 0
j:  0
Swap  Si: 00  with Sj:  00

i:  1
Si:  1
secret_key[i % keylen] 2
j:  3
Swap  Si: 01  with Sj:  03

i:  2
Si:  2
secret_key[i % keylen] 73
j:  78
Swap  Si: 02  with Sj:  4E

i:  3
Si:  1
secret_key[i % keylen] 0
j:  79
Swap  Si: 01  with Sj:  4F

i:  4
Si:  4
secret_key[i % keylen] 2
j:  85
Swap  Si: 04  with Sj:  55

i:  5
Si:  5
secret_key[i % keylen] 73
j:  163
Swap  Si: 05  with Sj:  A3

i:  6
Si:  6
secret_key[i % keylen] 0
j:  169
Swap  Si: 06  with Sj:  A9

i:  7
Si:  7
secret_key[i % keylen] 2
j:  178
Swap  Si: 07  with Sj:  B2

i:  8
Si:  8
secret_key[i % keylen] 73
j:  3
Swap  Si: 08  with Sj:  4F

i:  9
Si:  9
secret_key[i % keylen] 0
j:  12
Swap  Si: 09  with Sj:  0C

i:  10
Si:  10
secret_key[i % keylen] 2
j:  24
Swap  Si: 0A  with Sj:  18

i:  11
Si:  11
secret_key[i % keylen] 73
j:  108
Swap  Si: 0B  with Sj:  6C

i:  12
Si:  9
secret_key[i % keylen] 0
j:  117
Swap  Si: 09  with Sj:  75

i:  13
Si:  13
secret_key[i % keylen] 2

In [97]:
def rc4_decrypt(secret_key, encrypted_input):
    # Initialize S array
    S = [i for i in range(256)]
    j = 0
    keylen = len(secret_key)

    # KSA: Key Scheduling Algorithm
    for i in range(256):
        j = (j + S[i] + secret_key[i % keylen]) & 0xFF
        S[i], S[j] = S[j], S[i]

    print("Working Memory bytes after loop 1 (hex):")
    print([f"{b:02X}" for b in S])

    # PRGA: Pseudo-Random Generation Algorithm + decryption
    i = 0
    j = 0
    decrypted_output = []

    for k in range(len(encrypted_input)):
        i = (i + 1) & 0xFF
        print ("\ni: ",[f"{i:02X}"])
        j = (j + S[i]) & 0xFF
        print ("j: ",[f"{j:02X}"])
        print("Swap ","Si:", [f"{S[i]:02X}"] ," with Sj: ",[f"{S[j]:02X}"])
        S[i], S[j] = S[j], S[i]
        f_index = (S[i] + S[j]) & 0xFF
        print ("f_index: ",[f"{f_index:02X}"])
        f = S[f_index]
        print ("f: ",[f"{f:02X}"])
        print ("encrypted_input: ",[f"{encrypted_input[k]:02X}"])
        decrypted_byte = f ^ encrypted_input[k]
        print ("decrypted_byte: ",[f"{decrypted_byte:02X}"])
        decrypted_output.append(decrypted_byte)
    print("Working Memory bytes after loop 2 (hex):")
    print([f"{b:02X}" for b in S])
    return decrypted_output


In [98]:
# Secret key
secret_key = [0x00, 0x02, 0x49]

# Encrypted input from image (hex to int)
encrypted_input = [
    0x2D, 0x8F, 0x7A, 0xA9, 0x38, 0x73, 0x5F, 0x87,
    0x45, 0x1B, 0x82, 0x86, 0x4B, 0x9B, 0x7F, 0x9D,
    0xEF, 0x0D, 0xC4, 0xBB, 0xF9, 0x77, 0x99, 0x75,
    0xFF, 0xD5, 0x60, 0x73, 0x01, 0xF8, 0X16, 0x25,
]  # Exactly 32 bytes


# Decrypt
decrypted = rc4_decrypt(secret_key, encrypted_input)

# Display result
print("\nDecrypted Output (as ASCII):")
print(''.join(chr(b) for b in decrypted))

# Optional: Show hex output
print("\nDecrypted bytes (hex):")
print([f"{b:02X}" for b in decrypted])

Working Memory bytes after loop 1 (hex):
['3A', 'A8', '4E', '08', '55', 'A3', 'B8', '40', '93', '0C', '44', '24', 'A6', '89', '26', 'EA', '72', '66', '61', '5A', 'DA', 'EF', 'B2', '9E', '1F', '02', '15', '3D', '01', '8E', 'AC', 'CD', '4A', '57', '7B', '9B', '2C', '32', '49', 'C8', '79', '30', '64', '11', 'CC', '7E', '81', 'CE', '9D', '5C', '46', 'FA', '65', 'B1', '52', '2F', '74', 'C4', 'AB', '88', 'C0', 'FF', '86', 'F4', '16', '91', '3F', '0B', 'A5', '0F', 'CA', '90', '37', 'F3', 'E8', 'BE', '8F', '67', '09', '5F', 'AD', 'A4', 'EE', 'BF', 'A1', '8A', 'F2', 'EC', 'A0', '5E', '1E', '96', '45', 'C2', '3B', '28', '2B', '68', 'ED', '36', 'E5', '92', '9A', 'B3', 'DB', '77', '6A', 'D4', 'A2', '56', '27', '1B', 'EB', '54', '98', '84', '25', 'BC', '34', 'FB', '42', 'F0', '17', 'D0', 'D2', '13', '51', '4C', '33', 'C3', '1A', '31', 'F6', '60', '82', '10', 'E1', '73', '41', 'D8', '4B', '3C', 'DF', 'AA', '5D', '9C', '05', 'D6', '0A', '19', 'C9', '0E', 'FC', '06', '6D', 'F5', '99', '58', '29', 'B6'